In [4]:
import sys, re, json
from os import listdir,path, makedirs
from PIL import Image

with open('../elvconfig.json') as cfg_data:
    cfg = json.load(cfg_data)

columns,per_page = 1,7

####################################################################
inputpath = cfg['inputpath']
outputpath = cfg['outputpath']
startpage = cfg['startpage']



In [1]:
pat = r'.*\((.*\d)\).png'
items = [f for f in listdir(inputpath) if re.match(pat,f)]
items.sort(key=lambda f: int(''.join(list(filter(str.isdigit,f)))))
images = list(map(lambda item: Image.open(inputpath + item),items))

if not path.exists(outputpath):
    makedirs(outputpath)

#new_im = Image.new(mode='RGB', size=(total_width, max_height))
for im in images:
    idx = images.index(im)
    pagenum = int(idx//per_page) + startpage
    filename = '_(' + str(pagenum) + ').png'
    x_offset, y_offset = 0,0
    
    if idx % per_page:  
        #block for other 
        co = idx % columns
        
        #ro = int(idx%(per_page/columns))
        ro = int((idx // columns) % (per_page/columns))
        
        x_offset = im.size[0] * co
        y_offset = im.size[1] * ro
        
        new_im.paste(im,(x_offset,y_offset))
    else:
        #block whereby the first image in the group

        width, height = im.size
        total_width = width * columns
        max_height = height * int(per_page/columns)
        new_im = Image.new(mode='RGB', size=(total_width, max_height))
        #print('first')
        new_im.paste(im,(x_offset,y_offset))
    #print("IND:",idx, "X:",x_offset,"Y:",y_offset)
    if (idx % per_page) == (per_page-1):
        new_im.save(path.join(outputpath,filename),'PNG')
        print(idx % per_page)
print("Finished")

6
Finished
